In [190]:
import pandas as pd
import numpy as np
import re
import json

In [141]:
senate = pd.read_csv('senate_seat_forecast.csv')
senate = senate[senate['model'] == 'classic']
senate = senate[senate['forecastdate'] == '2018-10-29']
senate['location'] = senate['state'].map(str) + '-' + senate['class'].map(str)
senate = senate.drop(['state', 'class'], axis=1)

repulicans = senate[senate['party'] == 'R']
leans_republican = repulicans[(repulicans['win_probability'] >= 0.6) & (repulicans['win_probability'] < 0.95)]['location'].values

democrats = senate[senate['party'] == 'D']
leans_democrat = democrats[(democrats['win_probability'] >= 0.6) & (democrats['win_probability'] < 0.95)]['location'].values

low_probablities = senate[senate['win_probability'] < 0.6]
sum_locations = low_probablities.groupby('location').sum()
tossups = sum_locations[sum_locations['win_probability'] == 1]
tossups = tossups.index.values

required_locations = np.concatenate((leans_republican, leans_democrat, tossups), axis=0)
required_locations = senate[senate['location'].isin(required_locations)]

senate_required = required_locations[required_locations['win_probability'] != 0].reset_index()

senate_required.to_csv('senate_required.csv')

In [179]:
senate_candidates_only = senate_required['candidate'].str.lower().map(lambda x: re.sub(r'\W+', ' ', x))
senate_candidates_only.to_csv("senate_candidates.csv")

In [154]:
house = pd.read_csv('house_district_forecast.csv')
house = house[house['model'] == 'classic']
house = house[house['forecastdate'] == '2018-10-29']
house['location'] = house['state'].map(str) + '-' + house['district'].map(str)
house = house.drop(['state', 'district'], axis=1)

repulicans = house[house['party'] == 'R']
leans_republican = repulicans[(repulicans['win_probability'] >= 0.6) & (repulicans['win_probability'] < 0.75)]['location'].values

democrats = house[house['party'] == 'D']
leans_democrat = democrats[(democrats['win_probability'] >= 0.6) & (democrats['win_probability'] < 0.75)]['location'].values

low_probablities = senate[senate['win_probability'] < 0.6]
sum_locations = low_probablities.groupby('location').sum()
tossups = sum_locations[sum_locations['win_probability'] == 1]
tossups = tossups.index.values

required_locations = np.concatenate((leans_republican, leans_democrat, tossups), axis=0)
required_locations = house[house['location'].isin(required_locations)]

house_required = required_locations[required_locations['win_probability'] != 0].reset_index()

house_required.to_csv('house_required.csv')

In [180]:
house_candidates_only = house_required['candidate'].str.lower().map(lambda x: re.sub(r'\W+', ' ', x))
house_candidates_only.to_csv("house_candidates.csv")

In [217]:
manual_keywords = ["2018Midterms","2018MidtermElections","Election2018","ElectionDay","MAGA2018","MAGA","Trump2020","AmericaFirst","TheResistance","resist","VoteThemOut","2018Senate","VoteRed","Voteblue","WalkAwayFromDemocrats2018","WalkAwayFromRepublicans2018","VOTE","GAEarlyVoting","EarlyVoting","plus1","IVoted","WinBlue","WinRed","BlueWave","RedWave","republican2018","democrat2018","Republican","Democrat"]
house_keywords = list(house_candidates_only)
senate_keywords = list(senate_candidates_only)

In [218]:
keywords = {}

keywords['manual'] = {}
keywords['house'] = {}
keywords['senate'] = {}

keywords['manual']['start_date'] = "2018-10-28"
keywords['house']['start_date'] = "2018-10-28"
keywords['senate']['start_date'] = "2018-10-28"

keywords['manual']['keywords'] = str(manual_keywords).replace('"', '')
keywords['house']['keywords'] = str(house_keywords).replace('"', '')
keywords['senate']['keywords'] = str(senate_keywords).replace('"', '')

In [219]:
keywords

{'manual': {'start_date': '2018-10-28',
  'keywords': "['2018Midterms', '2018MidtermElections', 'Election2018', 'ElectionDay', 'MAGA2018', 'MAGA', 'Trump2020', 'AmericaFirst', 'TheResistance', 'resist', 'VoteThemOut', '2018Senate', 'VoteRed', 'Voteblue', 'WalkAwayFromDemocrats2018', 'WalkAwayFromRepublicans2018', 'VOTE', 'GAEarlyVoting', 'EarlyVoting', 'plus1', 'IVoted', 'WinBlue', 'WinRed', 'BlueWave', 'RedWave', 'republican2018', 'democrat2018', 'Republican', 'Democrat']"},
 'house': {'start_date': '2018-10-28',
  'keywords': "['don young', 'alyse s galvin', 'katie hill', 'steve knight', 'katie porter', 'mimi walters', 'harley rouda', 'dana rohrabacher', 'michael waltz', 'nancy soderberg', 'mario diaz balart', 'mary barzee flores', 'karen c handel', 'lucy mcbath', 'cindy axne', 'david young', 'mike bost', 'brendan kelly', 'rodney davis', 'betsy dirksen londrigan', 'randy hultgren', 'lauren underwood', 'jared golden', 'bruce poliquin', 'william lacy clay jr ', 'greg gianforte', 'kathl

In [208]:
with open('keywords.txt', 'w') as outfile:
    json.dump(keywords, outfile)
    
#manual modifications done